In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import scipy.stats as stats 
import numpy as np 
import statsmodels.api as sm 
from sklearn.cluster import KMeans 
import scikit_posthocs as sp 
from scipy.stats import kruskal 
from sklearn.utils import resample 
from sklearn.feature_selection import RFE 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score 
 
pd.set_option('display.max_columns', None)  # Отображение всех столбцов 
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк    
regions = pd.read_csv('regions.csv') 
starts = pd.read_csv('starts.csv') 
volunteers = pd.read_csv('volunteers.csv') 
 
 
 
volunteers = volunteers[volunteers.athlete_age < 100] 
starts = starts[starts.athlete_age < 80] 
starts = starts[starts.finish_time < 5000] 
 
starts['rain.1h'] = starts['rain.1h'].fillna(0) 
starts['snow.1h'] = starts['snow.1h'].fillna(0) 
 
cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg( 
    cnt_runners = ('athlete_id', 'count') 
) 
 
 
 
 
ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique() 
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique() 
common_athletes = set(ages_14) & set(ages_15) 
starts = starts[~starts['athlete_id'].isin(common_athletes)] 
starts = starts[starts['athlete_age'] > 14] 
 
 
starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0) 
 
 
volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)   
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')   
 
starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было 
 
print(len(starts)) 
unique_dates = ( 
    pd.concat([ 
        starts[['athlete_id', 'event_date']], 
        volunteers[['athlete_id', 'event_date']] 
    ]) 
    .drop_duplicates() 
) 
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count') 
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int) 
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left') 
 
 
# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]   
starts = starts[starts.event_date <= '2023-11-15'] 
 
 
 
newbie = starts[starts['is_newbie'] == 1] 
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id') 
didnt_come = first_runs[first_runs.is_repeated == 0] 
 
come = first_runs[first_runs.is_repeated == 1] 
come_with_didnt_come = pd.concat([come, didnt_come]) 
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1) 
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left") 
print(len(come_with_didnt_come))


166516
18114


In [4]:
comeback_newbies = come_with_didnt_come[come_with_didnt_come['is_repeated'] == 1]
comeback_newbies = comeback_newbies.iloc[:, 25:52]
comeback_newbies = comeback_newbies.drop(columns=['Координация волонтёров', 'Помощь в раздаче карточек позиций', 'Сурдопереводчик', 'Проведение разминки', 'Координатор парковки', 'is_repeated'])
comeback_newbies.to_csv('returned_newbies.csv')

In [5]:
comeback_newbies

,Завершение мероприятия,Замыкающий,Инструктаж новых участников,Лидер для слабовидящих,Маршал,Обработка результатов,Организатор,Организация финиша,Пейсер,Подготовка мероприятия,Проверка карточек позиций,Проверка трассы,Раздача карточек позиций,Разное,Связи с общественностью,Секундомер,Сканирование штрих-кодов,Сортировка карточек,Составление отчёта,Фотограф,Хранение и доставка оборудования
0,0.0,1.0,0.0,0.0,4.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,3.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,4.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,3.0,0.0
4,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9649,0.0,1.0,0.0,0.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
9650,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,2.0,1.0,0.0,1.0,1.0
9651,0.0,1.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
9652,0.0,1.0,0.0,0.0,4.0,1.0,2.0,3.0,1.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,3.0,2.0


In [8]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare, chi2

def friedman_test_all(df, alpha=0.05):
    """
    Рассчитывает критерий Фридмана для всех столбцов в DataFrame и определяет критическое значение.
    
    :param df: DataFrame, содержащий количественные данные.
    :param alpha: Уровень значимости для проверки гипотезы.
    :return: Значение статистики, критическое значение, p-value и вывод по гипотезе.
    """
    # Подготавливаем данные для теста
    samples = [df[col].values for col in df.columns]
    
    # Применяем критерий Фридмана
    stat, p_value = friedmanchisquare(*samples)
    
    # Число степеней свободы: k - 1, где k - количество групп (столбцов)
    k = len(df.columns)
    df_chi2 = k - 1  # Степени свободы
    
    # Критическое значение
    crit_value = chi2.ppf(1 - alpha, df_chi2)
    
    # Проверка гипотезы
    if stat > crit_value:
        result = "Различия есть"
    else:
        result = "Различий нет"
    
    return stat, crit_value, p_value, result

# Применение к таблице с количественными данными
alpha = 0.05  # Уровень значимости
stat, crit_value, p_value, result = friedman_test_all(comeback_newbies, alpha)

# Вывод результатов
print(f"Статистика критерия Фридмана: {stat:.4f}")
print(f"Критическое значение: {crit_value:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"Результат: {result}")


Статистика критерия Фридмана: 98424.4039
Критическое значение: 31.4104
p-value: 0.0000
Результат: Различия есть
